In [9]:
!pip install -q -U google-generativeai

In [10]:
import google.generativeai as genai


In [19]:
from kaggle_secrets import UserSecretsClient
import google.generativeai as genai
import os

user_secrets = UserSecretsClient()
my_key = user_secrets.get_secret("GOOGLE_API_KEY")

os.environ["GOOGLE_API_KEY"] = my_key
genai.configure(api_key=my_key)

In [20]:
from dataclasses import dataclass, field
from typing import List, Dict
from datetime import datetime
@dataclass
class Memory:
    messages: List[Dict] = field(default_factory=list)
    max_history: int = 20

    def add(self, role, content):
        self.messages.append({
            "role": role,
            "content": content,
            "time": datetime.now().isoformat()
        })
        # Keep memory within the limit
        if len(self.messages) > self.max_history:
            self.messages = self.messages[-self.max_history:]

    def get_context(self):
        # Formats the conversation history for the LLM
        out = ""
        for m in self.messages:
            out += f"{m['role']}: {m['content']}\n"
        return out

In [21]:
class IntentAgent:
    def classify(self, message):
        text = message.lower()
        
        # 1. Resume / CV Parsing
        if any(word in text for word in ["resume", "cv", "bio", "review"]):
            return "resume_review", "high"
        
        # 2. Learning Paths / Roadmaps
        if any(word in text for word in ["roadmap", "learn", "study", "course", "path"]):
            return "learning_path", "medium"
        
        # 3. Interview Preparation
        if any(word in text for word in ["interview", "mock", "questions", "behavioral"]):
            return "interview_prep", "high"
            
        # 4. General
        if "help" in text:
            return "general_help", "low"
            
        return "general", "low"

In [22]:
# Reply Agent ---
import google.generativeai as genai

class ReplyAgent:
    def __init__(self):
        self.model = genai.GenerativeModel('gemini-2.0-flash')

    def create_reply(self, message, intent, urgency, history_context):
        prompt = f"""
        You are a helpful Career Path AI Coach.
        
        CONVERSATION HISTORY:
        {history_context}
        
        CURRENT USER INPUT: "{message}"
        DETECTED INTENT: {intent} (Urgency: {urgency})
        
        INSTRUCTIONS:
        - If the intent is 'resume_review', provide constructive feedback or ask for the resume text.
        - If 'learning_path', suggest a brief, step-by-step roadmap (max 3 steps) for the topic.
        - If 'interview_prep', offer a mock question or a tip.
        - Be concise, encouraging, and professional.
        """
        
        try:
            response = self.model.generate_content(prompt)
            return response.text
        except Exception as e:
            return f"AI Error: {str(e)}"

In [23]:
class Coordinator:
    def __init__(self):
        self.intent_agent = IntentAgent()
        self.reply_agent = ReplyAgent()
        self.memory = Memory()

    def ask(self, message):
        # 1. Store User Message in Memory
        self.memory.add("user", message)
        
        # 2. Determine Intent (Agent 1)
        intent, urgency = self.intent_agent.classify(message)
        
        # 3. Retrieve Context from Memory
        context_str = self.memory.get_context()
        
        # 4. Generate Reply using LLM (Agent 2)
        # We pass 'context_str' here, which fixes the previous error
        reply = self.reply_agent.create_reply(message, intent, urgency, context_str)

        # 5. Store Agent Reply in Memory
        self.memory.add("agent", reply)
        
        return {
            "intent": intent,
            "reply": reply
        }

In [24]:
# Initialize the Career Agent
career_bot = Coordinator()

# Test Messages
messages = [
    "I need to learn Python for Data Science.",
    "Can you review my resume?",
    "I have a big interview coming up on Monday.",
    "Hi, I need career help."
]

print(f"{'USER INPUT':<45} | {'INTENT':<15} | {'REPLY'}")
print("-" * 110)

for msg in messages:
    out = career_bot.ask(msg)
    # Printing a truncated version of the reply to keep the table clean
    preview_reply = out['reply'][:60].replace('\n', ' ') + "..."
    print(f"{msg:<45} | {out['intent']:<15} | {preview_reply}")

USER INPUT                                    | INTENT          | REPLY
--------------------------------------------------------------------------------------------------------------
I need to learn Python for Data Science.      | learning_path   | Okay, great! Learning Python for Data Science is a fantastic...
Can you review my resume?                     | resume_review   | Okay, I can definitely help with that! To give you the best ...
I have a big interview coming up on Monday.   | interview_prep  | That's great you have an interview coming up! Let's get you ...
Hi, I need career help.                       | general_help    | Hi! I'm happy to help you navigate your career path. To give...
